In [1]:
! pip install transformers datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 13.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (4

# Question answering

## Load SQuAD dataset

In [2]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:5000]")
# squad = load_dataset("squad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [3]:
squad

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 5000
})

In [4]:
# squad['test'] = squad.pop('validation')

In [5]:
# squad

Split the dataset's `train` split into a train and test set with the [train_test_split](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.train_test_split) method:

In [6]:
squad = squad.train_test_split(test_size=0.2) # 5000 -> 4000 train, 1000 test

In [7]:
squad["train"][0]

{'id': '56cc803f6d243a140015efbf',
 'title': 'IPod',
 'context': 'Apple introduced a new 8-pin dock connector, named Lightning, on September 12, 2012 with their announcement of the iPhone 5, the fifth generation iPod Touch, and the seventh generation iPod Nano, which all feature it. The new connector replaces the older 30-pin dock connector used by older iPods, iPhones, and iPads. Apple Lightning cables have pins on both sides of the plug so it can be inserted with either side facing up.',
 'question': 'How many pins did the Lightning connector have?',
 'answers': {'text': ['8'], 'answer_start': [23]}}

There are several important fields here:

- `answers`: the starting location of the answer token and the answer text.
- `context`: background information from which the model needs to extract the answer.
- `question`: the question a model should answer.

## Preprocess

In [8]:
from transformers import AutoTokenizer

# tokenizer = None # TODO: load distilbert-base-uncased tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")  # TODO: load distilbert-base-uncased tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [17]:
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

There are a few preprocessing steps particular to question answering tasks you should be aware of:

1. Some examples in a dataset may have a very long `context` that exceeds the maximum input length of the model. To deal with longer sequences, truncate only the `context` by setting `truncation="only_second"`.
2. Next, map the start and end positions of the answer to the original `context` by setting
   `return_offset_mapping=True`.
3. With the mapping in hand, now you can find the start and end tokens of the answer. Use the [sequence_ids](https://huggingface.co/docs/tokenizers/main/en/api/encoding#tokenizers.Encoding.sequence_ids) method to
   find which part of the offset corresponds to the `question` and which corresponds to the `context`.

Here is how you can create a function to truncate and map the start and end tokens of the `answer` to the `context`:

In [9]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )
    # inputs --> has columns 'input_ids', 'attention_mask' , 'offset_mapping'

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        # start_char = None # TODO: find start character of answer
        # end_char = None # TODO: find end character of answer
        #  'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])
        # start_char = answer["answer_start"]
        # end_char = answer["answer_start"] + len(answer["text"])

        sequence_ids = inputs.sequence_ids(i) # [0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 ]
        # 0: is not answer, 1: is answer

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) function. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once. Remove any columns you don't need:

In [10]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)
# preprocess_function --> add columns 'input_ids', 'attention_mask' , 'start_positions', 'end_positions'
# remove_columns=squad["train"].column_names --> remove 'id', 'title', 'context', 'question', 'answers'

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [19]:
tokenized_squad # X: 'input_ids', 'attention_mask' , Y: 'start_positions', 'end_positions'

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 1000
    })
})

Now create a batch of examples using [DefaultDataCollator](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DefaultDataCollator). Unlike other data collators in 🤗 Transformers, the [DefaultDataCollator](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DefaultDataCollator) does not apply any additional preprocessing such as padding.

In [12]:
from transformers import DefaultDataCollator

# data_collator = None # TODO: make an instance
data_collator = DefaultDataCollator()

In [18]:
data_collator

DefaultDataCollator(return_tensors='pt')

## Train

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load DistilBERT with [AutoModelForQuestionAnswering](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForQuestionAnswering):

In [13]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

# model = None # TODO: load distilbert-base-uncased model
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")
# AutoModelForQuestionAnswering --> X: 'input_ids', 'attention_mask' , Y: 'start_positions', 'end_positions'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
model

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [14]:
training_args = TrainingArguments(
    output_dir="qa_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# TODO: pass the required arguments
# trainer = Trainer(
#     model= None,
#     args= None,
#     train_dataset= None,
#     eval_dataset= None,
#     tokenizer= None,
#     data_collator= None,
# )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"], # 4000 (X: 'input_ids', 'attention_mask' , Y: 'start_positions', 'end_positions')
    eval_dataset=tokenized_squad["test"], # 1000
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.239289
2,2.698000,1.680135
3,2.698000,1.622621


TrainOutput(global_step=750, training_loss=2.2687669677734377, metrics={'train_runtime': 486.334, 'train_samples_per_second': 24.674, 'train_steps_per_second': 1.542, 'total_flos': 1175877900288000.0, 'train_loss': 2.2687669677734377, 'epoch': 3.0})

In [15]:
# TODO: save both model and tokenizer
model.save_pretrained("qa_model")
tokenizer.save_pretrained("qa_model")
# trainer.save_model("qa_model")
# tokenizer.save_pretrained("qa_model")

('qa_model/tokenizer_config.json',
 'qa_model/special_tokens_map.json',
 'qa_model/vocab.txt',
 'qa_model/added_tokens.json',
 'qa_model/tokenizer.json')

In [33]:
from google.colab import userdata

# push to huggin face
USERNAME=userdata.get('HUGGINGFACE_USERNAME')
TOKEN=userdata.get('HUGGINGFACE_WRITE_ACCESS_TOKEN')
model.push_to_hub(f"{USERNAME}/distilbert_base_uncased_question_answering", use_auth_token=TOKEN, commit_message="Upload Question Answering Model")
tokenizer.push_to_hub(f"{USERNAME}/distilbert_base_uncased_question_answering", use_auth_token=TOKEN, commit_message="Upload Question Answering Tokenizer")

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:834: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/farzanrahmani/distilbert_base_uncased_question_answering/commit/f5a56a0803c2532dbc7d9c2565bb45123c986afb', commit_message='Upload Question Answering Tokenizer', commit_description='', oid='f5a56a0803c2532dbc7d9c2565bb45123c986afb', pr_url=None, pr_revision=None, pr_num=None)

## Evaluate

Evaluation for question answering requires a significant amount of postprocessing. To avoid taking up too much of your time, this guide skips the evaluation step. The [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) still calculates the evaluation loss during training so you're not completely in the dark about your model's performance.

If have more time and you're interested in how to evaluate your model for question answering, take a look at the [Question answering](https://huggingface.co/course/chapter7/7?fw=pt#postprocessing) chapter from the 🤗 Hugging Face Course!

## Inference

In [21]:
# question = None # TODO: write a question
# context = None # TODO: write a context for your question
question = "What is the capital of France?"
context = "Paris is the capital and most populous city of France. It is situated on the river Seine in northern France, at the heart of the Île-de-France region. The city of Paris, within its administrative limits largely unchanged since 1860, has an estimated population of 2,175,601 in 2018, but the Paris metropolitan area has a population of 12,995,809 across 800 square kilometers, most densely-populated area in Europe."
# question = "What does the fox do?"
# context = "The quick brown fox jumps over the lazy dog."

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for question answering with your model, and pass your text to it:

In [22]:
from transformers import pipeline

# question_answerer = None # TODO: call QA pipeline
question_answerer = pipeline("question-answering", model="qa_model", tokenizer="qa_model")
question_answerer(question=question, context=context)

{'score': 0.3740597367286682, 'start': 0, 'end': 5, 'answer': 'Paris'}

In [23]:
question_answerer

You can also manually replicate the results of the `pipeline` if you'd like:

Tokenize the text and return PyTorch tensors:

In [34]:
from transformers import AutoTokenizer

# tokenizer = None # TODO: load your tokenizer
# tokenizer = AutoTokenizer.from_pretrained("qa_model")
tokenizer = AutoTokenizer.from_pretrained(f"{USERNAME}/distilbert_base_uncased_question_answering")

inputs = tokenizer(question, context, return_tensors="pt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [25]:
inputs # all values in attention_mask are 1 because input was long and was truncated.
# if it was short it was padded and had zeros like [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]

{'input_ids': tensor([[  101,  2054,  2003,  1996,  3007,  1997,  2605,  1029,   102,  3000,
          2003,  1996,  3007,  1998,  2087, 20151,  2103,  1997,  2605,  1012,
          2009,  2003,  4350,  2006,  1996,  2314, 16470,  1999,  2642,  2605,
          1010,  2012,  1996,  2540,  1997,  1996, 17869,  1011,  2139,  1011,
          2605,  2555,  1012,  1996,  2103,  1997,  3000,  1010,  2306,  2049,
          3831,  6537,  4321, 15704,  2144,  7313,  1010,  2038,  2019,  4358,
          2313,  1997,  1016,  1010, 12862,  1010,  3438,  2487,  1999,  2760,
          1010,  2021,  1996,  3000,  4956,  2181,  2038,  1037,  2313,  1997,
          2260,  1010,  5585,  2629,  1010,  3770,  2683,  2408,  5385,  2675,
          7338,  1010,  2087, 19441,  1011, 10357,  2181,  1999,  2885,  1012,
           102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

Pass your inputs to the model and return the `logits`:

In [35]:
import torch
from transformers import AutoModelForQuestionAnswering

# model = None # TODO: load your model
# model = AutoModelForQuestionAnswering.from_pretrained("qa_model")
model = AutoModelForQuestionAnswering.from_pretrained(f"{USERNAME}/distilbert_base_uncased_question_answering")

# model.eval()
with torch.no_grad():
    # outputs = None # TODO: pass your inputs to the model
    outputs = model(**inputs)

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

In [27]:
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-2.8654, -3.5741, -4.7494, -4.2662, -4.0432, -5.5072, -4.4021, -5.1024,
         -5.3516,  4.4076, -3.3570, -0.9541, -0.9699, -3.4056, -0.9482, -0.8381,
          1.0475, -4.5986,  0.1835, -5.2236, -3.7382, -4.6155, -3.5044, -3.2778,
         -0.5807,  0.7959,  1.3312, -5.2502, -1.0498, -2.1382, -5.6315, -4.1427,
         -3.2573, -3.6305, -5.1506, -2.5245,  3.1458, -4.0104, -3.1438, -4.1355,
         -1.9292, -4.0858, -5.1852, -4.4827, -4.3406, -5.6209, -1.0957, -5.9889,
         -5.6502, -5.0763, -5.0990, -6.2808, -5.7015, -6.3169, -5.9046, -2.7004,
         -6.3822, -5.6023, -4.8429, -5.6749, -4.7586, -5.7157, -2.2301, -5.2284,
         -5.2250, -5.5104, -5.0310, -5.8352, -6.1501, -5.3758, -6.1927, -5.7566,
         -3.7068, -2.5860, -5.0302, -5.5841, -5.9819, -5.0056, -4.8970, -5.9021,
         -2.3315, -5.7243, -5.4769, -6.2995, -6.1050, -5.4947, -6.3463, -6.1243,
         -3.2484, -5.9449, -6.0945, -6.4650, -5.5429, -5

Get the highest probability from the model output for the start and end positions:

In [28]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [29]:
print(answer_start_index)
print(answer_end_index)

tensor(9)
tensor(9)


Decode the predicted tokens to get the answer:

In [30]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1] # input_ids[0, ...] -> batch = 0
# predict_answer_tokens --> 3000
tokenizer.decode(predict_answer_tokens)

'paris'

In [31]:
# another example
question = "What does the fox do?"
context = "The quick brown fox jumps over the lazy dog."
answer = question_answerer(question=question, context=context)

In [32]:
answer

{'score': 0.27648741006851196,
 'start': 4,
 'end': 43,
 'answer': 'quick brown fox jumps over the lazy dog'}